In [1]:
import numpy as np
import pandas as pd

infile = 'Prince.txt'
f = open(infile)
sourceInLine = f.readlines()
prince = ''
for line in sourceInLine:
    temp = line.strip('\n')
    prince  = prince + temp
cnt = 0
chars = ''
for letter in prince:
    cnt = max(1, len(chars))
    if letter >= 'a' and letter <= 'z' :
        chars = chars + letter
    elif letter >= 'A' and letter <= 'Z':
        chars = chars + chr(ord(letter) + 32)
    elif letter == ' ' and chars[cnt-1] != '{':
        chars = chars + '{' # chars # 将空格替换为{， ascll码连续性

list_letter = []  #字母列表
for i in range(26):
    list_letter.append(chr(97+i))
list_noblank = tuple(list_letter)
list_letter.append('{') # list_letter #字符列表


('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z')
26


#### 统计a-z的概率（不区分大小写）
出现概率最大的字符：空格，e，t，a，o

In [2]:
prob = np.zeros(27)
for letter in chars:  # ord(a) = 97
    prob[ord(letter) - 97] = prob[ord(letter) - 97] + 1
prob = prob / len(chars) # prior probability
list_prob = prob.tolist()
data1 = {'letter': list_letter, 'probability':list_prob}
df = pd.DataFrame(data1)
df

,letter,probability
0,a,0.064303
1,b,0.011298
2,c,0.016568
3,d,0.031490
4,e,0.105217
5,f,0.015647
6,g,0.014692
7,h,0.051935
8,i,0.057780
9,j,0.000874


#### 统计条件概率$P(a_i|a_j)$, 即发出前一个字母的条件下，后一个字母出现的概率
$P(a_i|a_j) = \frac{P(a_ja_i)}{P(a_j)}$ 
出现条件概率最大的字符序列 {w，ad，{y，ad

In [3]:
seq = []
cnt = 0
jprob = np.zeros(27*27)
cprob = np.zeros(27*27)
list_noblank
for j in list_letter:
    for i in list_letter:
        seq.append(j + i)
        jprob[cnt] = chars.count(j+i)
        cnt += 1
jprob = jprob / sum(jprob)  #joint probability

data2 = {'seq': seq, 'joint probability(%)': 100*jprob}
df2 = pd.DataFrame(data2)

cnt = 0
for j in list_letter:
    for i in list_letter:
        cprob[cnt] = jprob[cnt] / prob[ord(j)-97] # conditional probability
        cnt = cnt + 1


data3 = {'seq': seq, 'conditional probability(%)': 100*cprob}
df3 = pd.DataFrame(data3)
df3

,seq,conditional probability(%)
0,aa,0.000000
1,ab,2.129207
2,ac,1.914497
3,ad,4.186844
4,ae,0.035785
...,...,...
724,{w,6.176386
725,{x,0.000000
726,{y,2.492844
727,{z,0.005866


#### 若仅考虑字母的出现概率，不考虑字母间的关联，计算熵
$H = - \sum p(a_j)log_2p(a_j) = 4.05$

In [4]:
import math
H = 0
for p in prob:
    H -= p * math.log2(p)
print('Entropy', H)

Entropy 4.052183227489189


#### 若只考虑前后字母的关联，计算条件熵
$H_{joint} = -\sum P(a_j,a_i)log_2P(a_j,a_i)$

$H_{conditional} = -\sum P(a_j,a_i)log_2P(a_i|a_j)$

In [6]:
H_joint = 0
cnt = 0
for j in list_letter:
    for i in list_letter:
        if jprob[cnt] != 0:
            H_joint -= jprob[cnt] * math.log2(jprob[cnt])
        cnt += 1
print('joint entropy', H_joint)

H_conditional = 0
cnt = 0
for j in list_letter:
    for i in list_letter:
        if cprob[cnt] != 0:
            H_conditional -= jprob[cnt] * math.log2(cprob[cnt])
        cnt = cnt + 1
print('conditional entropy', H_conditional)

joint entropy 7.3068327639238495
conditional entropy 3.2546299512242065
